In [1]:
import pandas as pd
import pyodbc
from datetime import datetime
import time
import tqdm
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from datetime import datetime
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

F:\anaconda\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
server = 'DESKTOP-901337D\SQLEXPRESS'
db = 'StockMarket'
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + db + ';Trusted_COnnection=yes')
cursor = conn.cursor()

In [3]:
sql = 'SELECT DISTINCT Ticker FROM SecurityFundamentals order by Ticker'
tickers = pd.read_sql(sql,conn)

In [4]:
sql1 = 'SELECT DISTINCT Tag FROM SecurityFundamentals ORDER BY Tag'
TagUnic = pd.read_sql(sql1, conn)
print(TagUnic.shape)

(196, 1)


In [6]:
def calculate_quartile(Date):
    Date =  datetime.strptime(Date, '%Y-%m-%d').date()
    result = Date.year * 4 + (Date.month - 1) // 3
    return result

In [7]:
def monthdelta(date, delta):
    date =  datetime.strptime(date, '%Y-%m-%d').date()
    m, y = (date.month + delta) % 12, date.year + ((date.month) + delta - 1) // 12
    if not m: m = 12
    d = min(date.day, 
            [31, 29 if y%4==0 and not y%400==0 else 28,31,30,31,30,31,31,30,31,30,31][m-1]
           )
    return date.replace(day=d,month=m, year=y)

In [8]:
date = '2019-06-29'

In [9]:
def generate_new_features(date, num_of_quarts, important_tags):
    tickers_q = []
    features = []
    quart = calculate_quartile(date)
    qs = []
    for i in range(num_of_quarts):
        t = monthdelta(date, -3).strftime(format='%Y-%m-%d')
        qs.append(calculate_quartile(t))
        date = t
    for tick in (tickers['Ticker']):
        feature = []
        for tag in important_tags:
            values = []
            for q in qs:
                sql1 = "SELECT Value FROM SecurityFundamentals WHERE Ticker='" + tick + "' and Tag='" + tag + "' and q ='"+ str(q) +"'"
                tickPrices = pd.read_sql(sql1, conn)
                if(tickPrices.shape[0] == 0):
                    values.append('Nan')
                else:
                    values.append(tickPrices['Value'][0])
            for i in range(len(values) - 1):
                if values[i] == 'Nan' or values[i + 1] == 'Nan':
                    feature.append(-1)
                else:
                    feature.append(values[i] - values[i + 1])
        features.append(feature)
        tickers_q.append('{}_{}'.format(tick, quart))
    return (tickers_q, features)

In [10]:
%%time
labels_pd = pd.read_csv("labelsWithoutIndex.csv",sep=';')
labels_info = []
for index, row in tqdm.tqdm(labels_pd.iterrows()):
    q = int(row.Q)
    tick = row.Ticker
    label = int(row.label)
    labels_info.append(("{}_{}".format(tick, q), label))

127150it [00:13, 9644.78it/s] 


Wall time: 13.2 s


In [ ]:
%%time
labels = []
features = []
labels_pd = pd.read_csv("labels.csv")
features_pd = pd.read_csv("features.csv")
for index, row in features_pd.iterrows():
    features.append(row)
for index, row in labels_pd.iterrows():
    labels.append(row)

In [11]:
def stratify(labels):
    y = np.zeros(len(labels))
    i = 0
    while i < len(y):
        y[i] = 1
        i += 2
    return y

In [12]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [13]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'eta': [0.05, 0.1, 0.3],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5, 6, 9, 12],
        'n_estimators': [250, 500, 1000, 2000]
        }

In [15]:
num_of_quarts = [3,5,7,9]
important_tags = [['goodwill',
 'totalcurrentliabilities',
 'currentdeferredrevenue',
 'totalequity',
 'totalnoncurrentassets',
 'basiceps',
 'totaloperatingexpenses',
 'accountspayable',
 'incometaxexpense',
 'noncurrentdeferredrevenue',
 'cashandequivalents',
 'restructuringcharge',
 'totalpretaxincome',
 'netincomecontinuing',
 'totalassets',
 'incometaxexpense',
 'totalcommonequity',
 'currentdeferredrevenue',
 'operatingrevenue',
 'totalliabilitiesandequity'],
 ['goodwill',
 'totalcurrentliabilities',
 'currentdeferredrevenue',
 'totalequity',
 'totalnoncurrentassets'],
['goodwill',
 'totalcurrentliabilities',
 'currentdeferredrevenue',
 'totalequity',
 'totalnoncurrentassets',
 'basiceps',
 'totaloperatingexpenses',
 'accountspayable',
 'incometaxexpense',
 'noncurrentdeferredrevenue'],
['goodwill',
 'totalcurrentliabilities',
 'currentdeferredrevenue',
 'totalequity',
 'totalnoncurrentassets',
 'basiceps',
 'totaloperatingexpenses',
 'accountspayable',
 'incometaxexpense',
 'noncurrentdeferredrevenue',
 'cashandequivalents',
 'restructuringcharge',
 'totalpretaxincome',
 'netincomecontinuing',
 'totalassets'],
['goodwill',
 'totalcurrentliabilities',
 'currentdeferredrevenue',
 'totalequity',
 'totalnoncurrentassets',
 'basiceps',
 'totaloperatingexpenses',
 'accountspayable',
 'incometaxexpense',
 'noncurrentdeferredrevenue',
 'cashandequivalents',
 'restructuringcharge',
 'totalpretaxincome',
 'netincomecontinuing',
 'totalassets',
 'incometaxexpense',
 'totalcommonequity',
 'currentdeferredrevenue',
 'operatingrevenue',
 'totalliabilitiesandequity',
 'operatingcostofrevenue',
 'totalcostofrevenue',
 'totalrevenue',
 'treasurystock',
 'totalrevenue'],
['goodwill',
 'totalcurrentliabilities',
 'currentdeferredrevenue',
 'totalequity',
 'totalnoncurrentassets',
 'basiceps',
 'totaloperatingexpenses',
 'accountspayable',
 'incometaxexpense',
 'noncurrentdeferredrevenue',
 'cashandequivalents',
 'restructuringcharge',
 'totalpretaxincome',
 'netincomecontinuing',
 'totalassets',
 'incometaxexpense',
 'totalcommonequity',
 'currentdeferredrevenue',
 'operatingrevenue',
 'totalliabilitiesandequity',
 'operatingcostofrevenue',
 'totalcostofrevenue',
 'totalrevenue',
 'treasurystock',
 'totalrevenue',
 'intangibleassets',
 'netincomecontinuing',
 'weightedavedilutedsharesos',
 'othercurrentliabilities',
 'commonequity']] 

In [16]:
def count_of_zeros_and_ones(y_test):
    zeros = 0
    ones = 0
    for i in range(len(y_test)):
        if (y_test[i]== 0) :
            zeros += 1
        else:
            ones += 1
    return zeros,ones

In [17]:
min_acc = 30

In [18]:
for quarts in tqdm.tqdm(num_of_quarts):
    for imp_tags in tqdm.tqdm(important_tags):
        print("Num of quarts: {} Num of important tags: {}".format(quarts, len(imp_tags)))
        tickers_, features = generate_new_features(date, quarts,imp_tags)
        print('{}: {}'.format("Длина массива тикеров", len(tickers_))) 
        print('{}: {}'.format("Длина массива фичей", len(features)))
        print('{}: {}'.format("Кол-во фичей", len(features) * len(important_tags) * 2))
        # Из всех labels отберем только те, которые нам нужны
        labels_info_new = list(filter(lambda x: x[0] in tickers_, labels_info))
        labels = []
        for label in labels_info_new:
            labels.append(label[1])
        print('{}: {}'.format("Длина массива labels", len(labels)))
        # Разделим нашу выборку на обучающую и тестовую
        y = stratify(labels)
        X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33, random_state=55,stratify=y)
        # Преобразуем их в np.array для xgboost
        X_train = np.array(X_train)
        X_test = np.array(X_test)
        y_train = np.array(y_train)
        y_test = np.array(y_test)
        print('{}: {}'.format("X_train", len(X_train))) 
        print('{}: {}'.format("X_test", len(X_test))) 
        print('{}: {}'.format("y_train", len(y_train))) 
        print('{}: {}'.format("y_test", len(y_test))) 
        zeros,ones = count_of_zeros_and_ones(y_test)
        print("Кол-во нулей:{} \n Кол-во единиц: {}".format(zeros, ones))
        xgb1 = XGBClassifier(learning_rate=0.02, n_estimators=1000, metrics=['auc'], max_depth=3)
        folds = 10
        param_comb = 6
        skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)
        random_search = RandomizedSearchCV(xgb1, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X_train,y_train), verbose=3, random_state=1001 )
        # Here we go
        start_time = timer(None) # timing starts from this point for "start_time" variable
        random_search.fit(X_train, y_train)
        timer(start_time) # timing ends here for "start_time" variable
        print('\n All results:')
        print(random_search.cv_results_)
        print('\n Best estimator:')
        print(random_search.best_estimator_)
        print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
        print(random_search.best_score_ * 2 - 1)
        print('\n Best hyperparameters:')
        print(random_search.best_params_)
        y_pred = random_search.predict_proba(X_test)
        fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred[:,1])
        metrics.auc(fpr, tpr)
        accuracy = accuracy_score(y_test, y_pred[:, 1] > 0.5)
        print("Accuracy: %.2f%%" % (accuracy * 100.0))
            
        if (accuracy > min_acc):
            min_acc = accuracy
            Best_hyperparameters = random_search.best_params_
            quarts_best = quarts
            best_important_tags = imp_tags

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

Num of quarts: 3 Num of important tags: 20
Длина массива тикеров: 3644
Длина массива фичей: 3644
Кол-во фичей: 43728
Длина массива labels: 3644
X_train: 2441
X_test: 1203
y_train: 2441
y_test: 1203
Кол-во нулей:601 
 Кол-во единиц: 602
Fitting 10 folds for each of 6 candidates, totalling 60 fits

 Time taken: 0 hours 1 minutes and 47.21 seconds.

 All results:
{'mean_fit_time': array([ 2.45500922,  8.76671867,  7.30216534,  1.68488097, 17.35563087,
        3.61818376]), 'std_fit_time': array([0.27020087, 0.21022069, 0.14563642, 0.05215018, 0.19065293,
       0.13238248]), 'mean_score_time': array([0.00780787, 0.01917832, 0.01478858, 0.00352314, 0.04267583,
       0.01037035]), 'std_score_time': array([0.00780788, 0.00616638, 0.0012934 , 0.00616299, 0.00588278,
       0.00698753]), 'param_subsample': masked_array(data=[0.8, 0.8, 1.0, 0.6, 0.8, 0.6],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_n_estimators': maske


 17%|█████████████▎                                                                  | 1/6 [21:46<1:48:54, 1306.99s/it]

Num of quarts: 3 Num of important tags: 5
Длина массива тикеров: 3644
Длина массива фичей: 3644
Кол-во фичей: 43728
Длина массива labels: 3644
X_train: 2441
X_test: 1203
y_train: 2441
y_test: 1203
Кол-во нулей:601 
 Кол-во единиц: 602
Fitting 10 folds for each of 6 candidates, totalling 60 fits

 Time taken: 0 hours 0 minutes and 29.59 seconds.

 All results:
{'mean_fit_time': array([0.84777884, 2.92181144, 2.44915028, 0.56326337, 3.21740985,
       0.76072891]), 'std_fit_time': array([0.25954769, 0.04212972, 0.07421117, 0.02646758, 0.06196295,
       0.04662129]), 'mean_score_time': array([0.00359051, 0.01120052, 0.00549214, 0.00156236, 0.01538126,
       0.00352223]), 'std_score_time': array([0.00079768, 0.00243753, 0.00681732, 0.00468707, 0.00344794,
       0.00609651]), 'param_subsample': masked_array(data=[0.8, 0.8, 1.0, 0.6, 0.8, 0.6],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_n_estimators': masked_array


 33%|██████████████████████████▋                                                     | 2/6 [27:36<1:07:58, 1019.73s/it]

...


 67%|████████████████████████████████████████████████████                          | 4/6 [3:33:59<1:59:55, 3597.88s/it]

Num of quarts: 9 Num of important tags: 25
Длина массива тикеров: 3644
Длина массива фичей: 3644
Кол-во фичей: 43728
Длина массива labels: 3644
X_train: 2441
X_test: 1203
y_train: 2441
y_test: 1203
Кол-во нулей:601 
 Кол-во единиц: 602
Fitting 10 folds for each of 6 candidates, totalling 60 fits

 Time taken: 0 hours 7 minutes and 31.78 seconds.

 All results:
{'mean_fit_time': array([10.82385166, 38.90837917, 34.02800272,  7.72099874, 69.30708308,
       15.66810744]), 'std_fit_time': array([0.31149359, 0.15231184, 0.39639963, 0.09268342, 0.56376879,
       0.43873068]), 'mean_score_time': array([0.00744872, 0.01917937, 0.01474173, 0.00209441, 0.04008117,
       0.01296387]), 'std_score_time': array([0.00724558, 0.00500889, 0.00297235, 0.00329123, 0.00737402,
       0.00461981]), 'param_subsample': masked_array(data=[0.8, 0.8, 1.0, 0.6, 0.8, 0.6],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_n_estimators': maske


 83%|█████████████████████████████████████████████████████████████████             | 5/6 [4:57:33<1:07:02, 4022.85s/it]

Num of quarts: 9 Num of important tags: 30
Длина массива тикеров: 3644
Длина массива фичей: 3644
Кол-во фичей: 43728
Длина массива labels: 3644
X_train: 2441
X_test: 1203
y_train: 2441
y_test: 1203
Кол-во нулей:601 
 Кол-во единиц: 602
Fitting 10 folds for each of 6 candidates, totalling 60 fits

 Time taken: 0 hours 9 minutes and 0.16 seconds.

 All results:
{'mean_fit_time': array([13.00389621, 46.65509584, 40.955428  ,  9.3028651 , 81.87410376,
       18.80722508]), 'std_fit_time': array([0.32769383, 0.3384338 , 0.31405716, 0.11685896, 0.91113803,
       0.59908298]), 'mean_score_time': array([0.00731018, 0.01803792, 0.01695547, 0.01056972, 0.03915937,
       0.01417463]), 'std_score_time': array([0.00927591, 0.00761153, 0.00502982, 0.00647003, 0.00771114,
       0.0050635 ]), 'param_subsample': masked_array(data=[0.8, 0.8, 1.0, 0.6, 0.8, 0.6],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_n_estimators': masked


100%|██████████████████████████████████████████████████████████████████████████████| 4/4 [17:20:36<00:00, 15609.24s/it]
